In [1]:
import girder_client
gc=girder_client.GirderClient(apiUrl="http://transplant.digitalslidearchive.emory.edu:8080/api/v1")
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [2]:
import re
##dataMatrixOutput Yields accessionNumber and then two other fields; not sure what to call them
##dmSlideId will be the second field  dmField3 is the third field

kidneyDataMatrixRegex1 = "(?P<accessionNumber>S..-.*);(?P<dmSlideId>..);(?P<dmField3>.*)'"
k1 = re.compile(kidneyDataMatrixRegex1)
#m = k1.search("b'S15-555;S3;EMY8'")

# regexFormat1 = '.*?(?P<BrainID>FOX\d{1,2}[A-Z])? ?' + \
#                '(?P<Hemi>(LH MID|LH FRONTAL|LH FRONT|LH POST FRONTAL|CAUDAL|LH MISC|MISC|LH CAUDAL|LH))? ?' + \
#                'COL (?P<Column>\d{1,2}) S? ?(?P<Section>(\d{1,3}|MISC [A-Z])) ' + \
#                '(?P<Stain>(SILVER|NISSL|1:20K AVP|OXT 1:5000)) ' + \
#                '(?P<StainDate>(\d{1,2}/\d{1,2}/\d{1,2}|\d{1,2}/\d{1,2}))'

# metadataTags = [  # Metadata tags to capture (FOX_DATA example)
#     'BrainID', 'Column', 'Hemi',
#     'Section', 'Stain', 'StainDate']

# regex1 = re.compile(regexFormat1)



In [ ]:
unkDMOutput = {"accessionNumber":'UNK','dmSlideId':'UNK','dmField3':'UNK'}


for f in gc.listFolder('5cb8e389e6291400676076bf',parentFolderType='collection'):
    print("Scanning folder %s" % f['name'])
    itemSet = gc.listItem(f['_id'])

    for i in itemSet:
        if 'meta' in i:
            #print(i['name'],i['meta'])

            ### Delete non useful tags that are from the FOX stuff
            if 'BrainID' in i:
                gc.addMetadataToItem(i['_id'],{'BrainID':None, 'Section':None,'Hemi':None,'Column':None,'StainDate':None})
            ### Only run this if I don't have dmSlideId posted, since if I have that it means I already updated this
            if 'dataMatrixOutput' in i['meta']:
                if ('dmSlideId' not in i['meta']):
                    m = k1.search(i['meta']['dataMatrixOutput'])
                    if m:
                        print(m.groupdict())
                        gc.addMetadataToItem(i['_id'],m.groupdict())
                    else:
                        print("Regex didn't match for",i['meta'])
                        gc.addMetadataToItem(i['_id'],unkDMOutput)
        else:
            print("Found no data matrix output for this item",i['_id'])

        
        


In [3]:
##ocrRawText

ocrRawTextOutputs = []

for f in gc.listFolder('5cb8e389e6291400676076bf',parentFolderType='collection'):
    print("Scanning folder %s" % f['name'])
    itemSet = gc.listItem(f['_id'])

    for i in itemSet:
        if 'meta' in i:
            if 'ocrRawText' in i['meta']:
                ocrRawTextOutputs.append(i['meta']['ocrRawText'])

Scanning folder 2019-03-14
Scanning folder 2019-03-15
Scanning folder 2019-03-16
Scanning folder 2019-03-18
Scanning folder 2019-03-19
Scanning folder 2019-03-20
Scanning folder 2019-03-21
Scanning folder 2019-03-22
Scanning folder 2019-03-23
Scanning folder 2019-03-25
Scanning folder 2019-03-26
Scanning folder 2019-03-27
Scanning folder 2019-03-28
Scanning folder Bela
Scanning folder Tubulitis_anotation


In [17]:
%load_ext autoreload
%autoreload 2
import ocr

kdBxOcrRegex1 = "(?P<blockID>A1) (?P<stainID>L.|MT|PAS)"
# #"'.*(?P<SlideId>..).*(?P<stainKey>L.)'"
kd1 = re.compile(kdBxOcrRegex1)

partialMatch = noMatch = 0

for o in ocrRawTextOutputs:
    #print(o)
    
    co = ocr.clean_ocr_output(o)
    #print(co)
    m= kd1.search(co)
    if m:
#        print(m.groupdict())
        partialMatch+=1
    else:
        print(co)
        noMatch+=1
        
print("No Match:%d vs partial match: %d of %d"% (noMatch,partialMatch,len(ocrRawTextOutputs)))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 S1E7QUE A1 / U E5 KIDBX I ECHOLSGERRY T
5167006 A1 E M A KIDBX ECHOLSGERRY
8165131 A1 1 L3 : KIDBX DWUMFOURJERRY
8165131 A1 3 PAS : KIDBX I DWUMFOURJERRY
/ 5165131 MT KDBX DWUMFOUR IERRY
8165340 A1 IE L2 : KIDBX COPEKMBERELY
8164975 A1 F L1 KIDBX CHAPMANREBECCA
5164975 A1 I L2 : KIDBX CHAPMANREBECCA
 164369 UT CROWNER RICHARN
8164369 9: E KIDBX CROWDFRRICHARD
8164178 A1 KIDBX WATSONBRYANT
8164178 L2 KIDBX WATSONBRYANT
8164178 A1 I 11 L3 IF KIDBX WATSONBRYANT
16 4178 107920 2000 A1 1191 LI
8163457 A1 T MQU PARKERDOMINIC
8163457 A1 KIDBX PARKERDOMINIC
8163501 IIII KIDBX CHAPMANREBECCA
8163501 A1 G L3 I KIDBX CHAPMANREBECCA
8163501 1 AS JURY SHAPMANREBECCA
16 2387 106601 2000 A1 1 11/101 
8162292 PAS : KIDBX THOMAS KRISTINA
16 2292 10646 5 A 1 
0 S1 6221 3 A1 F L1 KIDBX / LOCKETTROSE
8162213 A1 KIDBX LOCKETIROSE
8162213 A1 2 PAS : KIDBX LOCKETTROSE

51615544 PAS R KIDBX I BROWNSANDRA
81614983 KIDBX DU